# OpenET DisALEXI
## Collection "Overpass" Examples

In [ ]:
import datetime
import pprint

import ee
import pandas as pd

from IPython.display import Image
import openet.disalexi as model

ee.Initialize()

In [ ]:
ndvi_palette = ['#EFE7E1', '#003300']
et_palette = [
    'DEC29B', 'E6CDA1', 'EDD9A6', 'F5E4A9', 'FFF4AD', 'C3E683', '6BCC5C', 
    '3BB369', '20998F', '1C8691', '16678A', '114982', '0B2C7A']

image_size = 768

### Input parameters

In [ ]:
collections = ['LANDSAT/LC08/C02/T1_L2']

et_reference_source = 'projects/openet/assets/reference_et/california/cimis/daily/v1'
et_reference_band = 'eto'
et_reference_factor = 1.0
et_reference_resample = 'bilinear'
# et_reference_source = 'IDAHO_EPSCOR/GRIDMET'
# et_reference_band = 'eto'
# et_reference_factor = 0.85
# et_reference_resample = 'bilinear'

# Date range you want to aggregate ET over
# End date is inclusive (like filterDate() calls)
start_date = '2017-07-01'
end_date = '2017-08-01'

# Only keep images with an average cloud cover less than
# Cloud cover filter parameter is not being passed in (yet)
cloud_cover = 70

# Number of extra days (at start and end) to include in interpolation
interp_days = 32
# Interpolation method - currently only LINEAR is supported
interp_method = 'LINEAR'

interp_args = {
    'interp_source': et_reference_source,
    'interp_band': et_reference_band,
    'interp_resample': et_reference_factor,
    # 'interp_factor': et_reference_factor,
}

test_xy = [-121.5265, 38.7399]
test_point = ee.Geometry.Point(test_xy)

study_area = ee.Geometry.Rectangle(
    test_xy[0] - 0.08, test_xy[1] - 0.04, 
    test_xy[0] + 0.08, test_xy[1] + 0.04)
study_region = study_area.bounds(1, 'EPSG:4326')
study_crs = 'EPSG:32610'

# # Hard code the study area and CRS
# study_area = ee.Geometry.Rectangle(-122.00, 38.50, -121.00, 39.0)
# study_region = study_area.bounds(1, 'EPSG:4326').coordinates().getInfo()
# study_crs = 'EPSG:32610'

### Landsat 8 Collection 2 SR Image Collection

In [ ]:
model_obj = model.Collection(
    collections=collections,
    start_date=start_date,
    end_date=end_date,
    geometry=test_point,
    cloud_cover_max=70,
    model_args={
        'et_reference_source': et_reference_source, 
        'et_reference_band': et_reference_band,
        'et_reference_factor': et_reference_factor,
        'et_reference_resample': et_reference_resample,
    },
    # filter_args={},
)
pprint.pprint(model_obj.get_image_ids())

In [ ]:
def get_region_df(info):
    """Convert the output of getRegions to a pandas dataframe"""
    col_dict = {}
    info_dict = {}
    for i, k in enumerate(info[0][4:]):
        col_dict[k] = i+4
        info_dict[k] = {}
        
    for row in info[1:]:
        date = datetime.datetime.utcfromtimestamp(row[3] / 1000.0).strftime('%Y-%m-%d')
        for k, v in col_dict.items():
            info_dict[k][date] = row[col_dict[k]]
            
    return pd.DataFrame.from_dict(info_dict)

## "Overpass" Collection

In [ ]:
overpass_coll = model_obj.overpass(variables=['ndvi', 'et', 'et_reference', 'et_fraction'])

In [ ]:
overpass_df = get_region_df(overpass_coll.getRegion(test_point, scale=30).getInfo())
pprint.pprint(overpass_df)
print('')
pprint.pprint(overpass_df[['et', 'et_reference']].sum())

### NDVI

In [ ]:
image_url = ee.Image(overpass_coll.select(['ndvi']).mean())\
    .reproject(crs=study_crs, scale=30)\
    .getThumbURL({'min': -0.1, 'max': 0.9, 'palette': ','.join(ndvi_palette), 
                  'region': study_region, 'dimensions': image_size})
Image(url=image_url, embed=True, format='png')

### ET fraction

In [ ]:
image_url = ee.Image(overpass_coll.select(['et_fraction']).mean())\
    .reproject(crs=study_crs, scale=30)\
    .getThumbURL({'min': 0.0, 'max': 1.2, 'palette': ','.join(et_palette), 
                  'region': study_region, 'dimensions': image_size})
Image(url=image_url, embed=True, format='png')

### Reference ET (from CIMIS)

In [ ]:
image_url = ee.Image(overpass_coll.select(['et_reference']).mean())\
    .reproject(crs=study_crs, scale=30)\
    .getThumbURL({'min': 0.0, 'max': 12, 'palette': ','.join(et_palette), 
                  'region': study_region, 'dimensions': image_size})
Image(url=image_url, embed=True, format='png')

### ET

In [ ]:
image_url = ee.Image(overpass_coll.select(['et']).mean())\
    .reproject(crs=study_crs, scale=30)\
    .getThumbURL({'min': 0.0, 'max': 12, 'palette': ','.join(et_palette), 
                  'region': study_region, 'dimensions': image_size})
Image(url=image_url, embed=True, format='png')